In [ ]:
This tutorial demonstrates basic usage of the PyMGRIT package. Our goal is solving Dahlquist’s test problem,

$$
u' = \lambda u \;\;\text{ in } (0, 5] \text{ with }\; u(0) = 1
$$

discretized by Backward Euler. To accomplish this, this tutorial will go through the following tasks:

1. Writing the vector class holding all time-dependent information
2. Writing the application class holding any time-independent data
3. Solving the problem
4. Looking at results

Then, we define the class *VectorDahlquist* containing a scalar member variable *value*:

In [ ]:
class VectorDahlquist(Vector):
    """
    Vector class for Dahlquist's test equation
    """

    def __init__(self, value):
        super().__init__()
        self.value = value

Furthermore, we must define the following seven member functions: *set\_values*, *get\_values*, *clone*, *clone_zero*, *clone_rand*, *\_\_add\_\_*, *\_\_sub\_\_*, *\_\_mul\_\_*, *norm*, *pack* and *unpack*.

The function *set\_values* receives data values and overwrites the values of the vector data and *get\_values* returns the vector data. For our class *VectorDahlquist*, the vector data is the scalar member variable value:

In [ ]:
def set_values(self, value):
    self.value = value

def get_values(self):
    return self.value

The function *clone* clones the object. The function *clone_zero* returns a vector object initialized with zeros; *clone_rand* similarly returns a vector object initialized with random data. For our class *VectorDahlquist*, these member functions are defined as follows:

In [ ]:
def clone(self):
    return VectorDahlquist(self.value)

def clone_zero(self):
    return VectorDahlquist(0)

def clone_rand(self):
    return VectorDahlquist(np.random.rand(1)[0])

The functions *\_\_add\_\_*, *\_\_sub\_\_*, *\_\_mul\_\_*, and *norm* define the addition and subtraction of two vector objects and the norm of a vector object, respectively. For our class *VectorDahlquist*, adding or subtracting two vector objects means adding or subtracting the values of the member variable *value* by using the functions *get_values* and *set_values*. The multiplication defines the multiplication of a vector objects with a float. We define the norm of a vector object as the norm (from *numpy*) of the member variable *value*:

In [ ]:
def __add__(self, other):
    tmp = VectorDahlquist(0)
    tmp.set_values(self.get_values() + other.get_values())
    return tmp

def __sub__(self, other):
    tmp = VectorDahlquist(0)
    tmp.set_values(self.get_values() - other.get_values())
    return tmp

def __mul__(self, other):
    tmp = VectorDahlquist(0)
    tmp.set_values(self.get_values() * other)
    return tmp

def norm(self):
    return np.linalg.norm(self.value)

The functions *pack* and *unpack* define the data to be communicated and how data is unpacked after receiving it. For our class *VectorDahlquist*, packing means setting the data to be communicated to the member variable *value* and unpacking means setting the member variable *value* to the received scalar value:

In [ ]:
def pack(self):
    return self.value

def unpack(self, value):
    self.value = value

## Summary:

The vector class must inherit from PyMGRIT’s core *Vector* class.

Member variables hold all data of a single time point.

The following member functions must be defined:

* *set_values* : Setting vector data
* *get_values* : Getting vector data
* *clone* : Initialization of vector data with equivalent values
* *clone_zero* : Initialization of vector data with zeros
* *clone_rand* : Initialization of vector data with random values
* *\_\_add\_\_* : Addition of two vector objects
* *\_\_sub\_\_* : Subtraction of two vector objects
* *\_\_mul\_\_* : Multiplication of a vector object with a float
* *norm* : Norm of a vector object (for measuring convergence)
* *pack* : Specifying communication data
* *unpack* : Unpacking communication data

In [ ]:
import numpy as np
from pymgrit.core.vector import Vector

class VectorDahlquist(Vector):
    """
    Vector class for Dahlquist's test equation
    """

    def __init__(self, value):
        super().__init__()
        self.value = value

    def set_values(self, value):
        self.value = value

    def get_values(self):
        return self.value

    def clone(self):
        return VectorDahlquist(self.value)

    def clone_zero(self):
        return VectorDahlquist(0)

    def clone_rand(self):
        return VectorDahlquist(np.random.rand(1)[0])

    def __add__(self, other):
        tmp = VectorDahlquist(0)
        tmp.set_values(self.get_values() + other.get_values())
        return tmp

    def __sub__(self, other):
        tmp = VectorDahlquist(0)
        tmp.set_values(self.get_values() - other.get_values())
        return tmp

    def __mul__(self, other):
        tmp = VectorDahlquist(0)
        tmp.set_values(self.get_values() * other)
        return tmp

    def norm(self):
        return np.linalg.norm(self.value)

    def pack(self):
        return self.value

    def unpack(self, value):
        self.value = value

# Application class

In the next step we write the application class that contains information about the problem we want to solve. Every application class must inherit from PyMGRIT’s core *Application* class.

For our test problem, we import PyMGRIT’s core *Application* class:

In [ ]:
from pymgrit.core.application import Application

Then, we define the class *Dahlquist* containing the member variable *vector_template* that defines the data structure for any user-defined time point as well as the member variable *vector_t_start* that holds the initial condition at time *t_start*:

In [ ]:
class Dahlquist(Application):
    """
    Application class for Dahlquist's test equation,
       u' = lambda u,  u(0) = 1,
    with lambda = -1
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # Set the data structure for any user-defined time point
        self.vector_template = VectorDahlquist(0)

        # Set the initial condition
        self.vector_t_start = VectorDahlquist(1)

Note: The time interval of the problem is defined in the superclass *Application*. This PyMGRIT core class contains the following member variables:

* t\_start : start time (left bound of time interval)
* t\_end : end time (right bound of time interval)
* nt : number of time points

Furthermore, we must define the time integration routine as the member function step that evolves a vector *u_start* from time *t_start* to time *t_stop*. For our test problem, we take a backward Euler step:

In [ ]:
def step(self, u_start: VectorDahlquist, t_start: float, t_stop: float) -> VectorDahlquist:
    z = (t_stop - t_start) * -1  # Note: lambda = -1
    tmp = 1 / (1 - z) * u_start.get_values()
    return VectorDahlquist(tmp)

## Summary

The application class must inherit from PyMGRIT’s core *Application* class.

The application class contains information about the problem we want to solve.

The application class must contain the following member variables and member functions:

* Variable *vector_template* : Data structure for any user-defined time point
* Variable *vector_t_start* : Holds the initial condition (same data structur as *vector_template*)
* Function *step* : Time integration routine

In [ ]:
# Import superclass Application
from pymgrit.core.application import Application

class Dahlquist(Application):
    """
    Application class for Dahlquist's test equation,
       u' = lambda u,  u(0) = 1,
    with lambda = -1
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # Set the data structure for any user-defined time point
        self.vector_template = VectorDahlquist(0)

        # Set the initial condition
        self.vector_t_start = VectorDahlquist(1)

    # Time integration routine
    def step(self, u_start: VectorDahlquist, t_start: float, t_stop: float) -> VectorDahlquist:
        z = (t_stop - t_start) * -1  # Note: lambda = -1
        tmp = 1 / (1 - z) * u_start.get_values()
        return VectorDahlquist(tmp)

# Solving the problem

The third step is to set up an MGRIT solver for the test problem.

First, import PyMGRIT:

In [ ]:
from pymgrit import *

Create Dahlquist's test problem for the time interval [0, 5] with 101 equidistant time points (100 time points + 1 time point for the initial time t = 0)  as an object of our application class *Dahlquist*:

In [ ]:
dahlquist = Dahlquist(t_start=0, t_stop=5, nt=101)

Construct a multigrid hierarchy for the test problem dahlquist using PyMGRIT’s core function *simple_setup_problem*:

In [ ]:
dahlquist_multilevel_structure = simple_setup_problem(problem=dahlquist, level=2, coarsening=2)

This tells PyMGRIT to set up a hierarchy with two temporal grid levels using the test problem *dahlquist* and a temporal coarsening factor of two, i.e., on the fine grid, the number of time points is 101, and on the coarse grid, 51 (=100/2+1) time points are used.

Set up the MGRIT solver for the test problem using *dahlquist_multilevel_structure* and set the solver tolerance to 1e-10:

In [ ]:
mgrit = Mgrit(problem=dahlquist_multilevel_structure, tol=1e-10)

Finally, solve the test problem using the *solve()* routine of the solver *mgrit*:

In [ ]:
info = mgrit.solve()

and returns the residual history, setup time, and solve time in dictionary *info* with the following key values:

* *conv* : residual history (2-norm of the residual at each iteration)
* *time_setup* : setup time [in seconds]
* *time_solve* : solve time [in seconds]

# Looking at results

The last step is to look at the results of our PyMGRIT run.

In the default setting,

* PyMGRIT’s core routine Mgrit() prints out the setup time.
* The solve() routine
    * prints out the residual history, along with convergence factors and runtimes, and
    * returns the residual history, setup time, and solve time.

For our example, we can plot the residuals as follows: First, we import *numpy* and *pyplot*:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Then, we get the residuals from the dictionary *info*:

In [ ]:
res = info['conv']

and plot the residuals:

In [ ]:
iters = np.arange(1, res.size+1)
plt.semilogy(iters, res, 'o-')
plt.xticks(iters)
plt.xlabel('iter #')
plt.ylabel('residual norm')
plt.show()